<a href="https://colab.research.google.com/github/Mahmuda-104/Thesis/blob/main/CNN_All_Filter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import os
from scipy.io import loadmat

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data_path = '/content/drive/My Drive/Data'

In [4]:
import os
import scipy.io
import numpy as np
from scipy import signal

lowcut = 0.5 
highcut = 100  
fs_original = 500
fs_new = 250

control_data = []
control_path = os.path.join(data_path, 'Control')
for foldername in os.listdir(control_path):
    subfolder_path = os.path.join(control_path, foldername)
    control_data.append(subfolder_path)

Control30 = []
for control_data_path in control_data:
    n_epochs = 55 #len([f for f in os.listdir(control_data_path) if f.endswith('.mat')])
    control_arr = []
    for i in range(1, n_epochs+1):
        epoch_path = f"{control_data_path}/trial{i}.mat"
        mat_data = scipy.io.loadmat(epoch_path)
        mat_data1 = mat_data["trialData_i"]
        
        # Average referencing
        average_potential = np.mean(mat_data1, axis=0, keepdims=True)
        referenced_data = mat_data1 - average_potential
        
        # Bandpass filtering
        b, a = signal.butter(4, [lowcut, highcut], fs=fs_original, btype='band')
        filtered_data = signal.filtfilt(b, a, referenced_data, axis=-1)
        
        # Apply notch filter
        f0 = 60  # Frequency to be removed (e.g., power line interference)
        Q = 30   # Quality factor
        w0 = f0 / (fs_original / 2)
        b, a = signal.iirnotch(w0, Q)
        #filtered_data = signal.filtfilt(b, a, filtered_data, axis=-1)

        # Downsampling
        num_samples_original = filtered_data.shape[-1]
        num_samples_new = int(num_samples_original * fs_new / fs_original)
        downsampled_data = signal.resample(filtered_data, num_samples_new, axis=-1)
        
        control_arr.append(np.array(downsampled_data))
        
    Control30.append(np.array(control_arr))

Control30 = np.array(Control30)
print(Control30[0].shape)


(55, 60, 1000)


In [5]:
print("Filtered and downsampled data shape:", Control30.shape)

Filtered and downsampled data shape: (32, 55, 60, 1000)


In [6]:
import os
import scipy.io
import numpy as np
from scipy import signal

lowcut = 0.5 
highcut = 100  
fs_original = 500
fs_new = 250  

concussed_data = []
concussed_path = os.path.join(data_path, 'Concussed')
for foldername in os.listdir(concussed_path):
    subfolder_path = os.path.join(concussed_path, foldername)
    concussed_data.append(subfolder_path)

Concussed52 = []
for concussed_data_path in concussed_data:
    n_epochs = 55 #len([f for f in os.listdir(control_data_path) if f.endswith('.mat')])
    concussed_arr = []
    for i in range(1, n_epochs+1):
        epoch_path = f"{concussed_data_path}/trial{i}.mat"
        mat_data = scipy.io.loadmat(epoch_path)
        mat_data1 = mat_data["trialData_i"]
        
        # Average referencing
        average_potential = np.mean(mat_data1, axis=0, keepdims=True)
        referenced_data = mat_data1 - average_potential
        
        # Bandpass filtering
        b, a = signal.butter(4, [lowcut, highcut], fs=fs_original, btype='band')
        filtered_data = signal.filtfilt(b, a, referenced_data, axis=-1)
        
        # Apply notch filter
        f0 = 60  # Frequency to be removed (e.g., power line interference)
        Q = 30   # Quality factor
        w0 = f0 / (fs_original / 2)
        b, a = signal.iirnotch(w0, Q)
        #filtered_data = signal.filtfilt(b, a, filtered_data, axis=-1)

        # Downsampling
        num_samples_original = filtered_data.shape[-1]
        num_samples_new = int(num_samples_original * fs_new / fs_original)
        downsampled_data = signal.resample(filtered_data, num_samples_new, axis=-1)
        
        concussed_arr.append(np.array(downsampled_data))
        
    Concussed52.append(np.array(concussed_arr))

Concussed52 = np.array(Concussed52)
print(Concussed52[0].shape)

(55, 60, 1000)


In [7]:
print("Filtered and downsampled data shape:", Concussed52.shape)

Filtered and downsampled data shape: (52, 55, 60, 1000)


In [8]:
import numpy as np
from sklearn.model_selection import train_test_split
dataset = np.concatenate((Control30, Concussed52), axis=0)

#(0 for control, 1 for concussed)
labels = np.concatenate((np.zeros(len(Control30)), np.ones(len(Concussed52))))

In [9]:
train_ratio = 0.8  # 80% for training
val_ratio = 0.1   # 10% for validation
test_ratio = 0.1  # 10% for testing

In [10]:
X_train, X_test, y_train, y_test = train_test_split(dataset, labels, test_size=test_ratio, stratify=labels, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=val_ratio/(train_ratio+val_ratio), stratify=y_train, random_state=42)

In [13]:
import tensorflow as tf
from tensorflow.keras import layers
from keras.callbacks import EarlyStopping

# Normalize the input data (if required)
# X_train = X_train / 255.0
# X_val = X_val / 255.0
# X_test = X_test / 255.0

# Reshape the input data based on the expected input shape of the model
input_shape = X_train.shape[1:]  # Assuming the first dimension is the number of samples
X_train = X_train.reshape((-1,) + input_shape)
X_val = X_val.reshape((-1,) + input_shape)
X_test = X_test.reshape((-1,) + input_shape)

# Build the CNN model
model = tf.keras.Sequential()
model.add(layers.Conv2D(16, kernel_size=(6, 6), strides=2, activation='relu', padding='same', input_shape=input_shape))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Dropout(0.3))
model.add(layers.Conv2D(32, kernel_size=(3, 3), activation='relu', strides=1, padding='same'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Dropout(0.3))
model.add(layers.Conv2D(64, kernel_size=(3, 3), activation='relu', strides=1, padding='same'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Dropout(0.3))
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(2, activation='softmax'))

learning_rate = 0.005  # Specify your desired learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Define the early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model with early stopping
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, batch_size=32, callbacks=[early_stopping])

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
val_loss, val_accuracy = model.evaluate(X_val, y_val)

# Print the test accuracy
print("Test Accuracy:", test_accuracy)

Epoch 1/100
3/3 [==============================] - 18s 4s/step - loss: 752.6311 - accuracy: 0.4545 - val_loss: 4.4868 - val_accuracy: 0.2222
Epoch 2/100
3/3 [==============================] - 12s 3s/step - loss: 56.6405 - accuracy: 0.5758 - val_loss: 3.0324 - val_accuracy: 0.6667
Epoch 3/100
3/3 [==============================] - 12s 3s/step - loss: 40.7103 - accuracy: 0.5909 - val_loss: 2.1152 - val_accuracy: 0.4444
Epoch 4/100
3/3 [==============================] - 13s 4s/step - loss: 12.1615 - accuracy: 0.4545 - val_loss: 2.3147 - val_accuracy: 0.5556
Epoch 5/100
3/3 [==============================] - 12s 3s/step - loss: 11.3363 - accuracy: 0.5152 - val_loss: 2.1384 - val_accuracy: 0.6667
Epoch 6/100
3/3 [==============================] - 12s 4s/step - loss: 10.5987 - accuracy: 0.5909 - val_loss: 0.8127 - val_accuracy: 0.4444
Epoch 7/100
3/3 [==============================] - 14s 5s/step - loss: 14.7188 - accuracy: 0.5455 - val_loss: 1.0128 - val_accuracy: 0.2222
Epoch 8/100
3/3 [==

In [12]:
import tensorflow as tf
from tensorflow.keras import layers
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

# Split the dataset into training, validation, and test sets
# X_train, X_test, y_train, y_test = train_test_split(dataset, labels, test_size=test_ratio, stratify=labels, random_state=42)
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=val_ratio/(train_ratio+val_ratio), stratify=y_train, random_state=42)

# Normalize the input data (if required)
# X_train = X_train / 255.0
# X_val = X_val / 255.0
# X_test = X_test / 255.0

# Reshape the input data based on the expected input shape of the model
input_shape = X_train.shape[1:]  # Assuming the first dimension is the number of samples
X_train = X_train.reshape((-1,) + input_shape)
X_val = X_val.reshape((-1,) + input_shape)
X_test = X_test.reshape((-1,) + input_shape)

# Build the CNN model
model = tf.keras.Sequential()
model.add(layers.Conv2D(16, kernel_size=(6, 6),strides=2, activation='relu',padding='same', input_shape=input_shape))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

# Compile the model with the desired loss function and optimizer
learning_rate = 0.005  # Specify your desired learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
# Train the model
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, batch_size=32, callbacks=[early_stopping])

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
val_loss, val_accuracy = model.evaluate(X_val, y_val)

# Print the test accuracy
print("Test Accuracy:", test_accuracy)

Epoch 1/100
3/3 [==============================] - 14s 4s/step - loss: 111.7407 - accuracy: 0.3939 - val_loss: 35.7130 - val_accuracy: 0.6667
Epoch 2/100
3/3 [==============================] - 15s 4s/step - loss: 124.5995 - accuracy: 0.6212 - val_loss: 25.0151 - val_accuracy: 0.3333
Epoch 3/100
3/3 [==============================] - 12s 3s/step - loss: 20.8125 - accuracy: 0.3939 - val_loss: 2.6317 - val_accuracy: 0.4444
Epoch 4/100
3/3 [==============================] - 12s 3s/step - loss: 9.0900 - accuracy: 0.7121 - val_loss: 5.6910 - val_accuracy: 0.2222
Epoch 5/100
3/3 [==============================] - 12s 3s/step - loss: 1.9878 - accuracy: 0.6818 - val_loss: 7.7669 - val_accuracy: 0.3333
Epoch 6/100
3/3 [==============================] - 12s 4s/step - loss: 1.5242 - accuracy: 0.7576 - val_loss: 9.2853 - val_accuracy: 0.2222
Epoch 7/100
3/3 [==============================] - 12s 4s/step - loss: 1.0393 - accuracy: 0.8030 - val_loss: 9.3076 - val_accuracy: 0.2222
Epoch 8/100
3/3 [===